In [8]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


# =====================================================
# STEP 1 — LOAD DATA
# =====================================================

train_path = "/content/drive/MyDrive/ML_Project/MultinomialClassification/Dataset/train.csv"
test_path  = "/content/drive/MyDrive/ML_Project/MultinomialClassification/Dataset/test.csv"



df_raw = pd.read_csv(train_path)
test_raw = pd.read_csv(test_path)

TARGET = "spend_category"
ID_COL = "trip_id"

print("Original Training Shape:", df_raw.shape)
print("Original Test Shape:", test_raw.shape)


# =====================================================
# HELPER — Convert Range (15-30, 90+) to numeric
# =====================================================
def range_to_mid(x):
    x = str(x).strip()
    if x.lower() in ["none", "", "nan", "null"]:
        return np.nan
    if "+" in x:
        return float(x.replace("+", ""))
    if "-" in x:
        a, b = x.split("-")
        return (float(a) + float(b)) / 2
    try:
        return float(x)
    except:
        return np.nan


# =====================================================
# STEP 2 — GLOBAL COLUMN DEFINITIONS
# =====================================================
binary_cols = [
    "is_first_visit","intl_transport_included","accomodation_included",
    "food_included","domestic_transport_included","sightseeing_included",
    "guide_included","insurance_included"
]

categorical_cols = [
    "country","age_group","travel_companions","main_activity",
    "visit_purpose","tour_type","info_source","arrival_weather"
]

numeric_count_cols = ["num_females","num_males","mainland_stay_nights","island_stay_nights"]


# =====================================================
# STEP 3 — REMOVE NULL TARGETS FIRST
# =====================================================
removed_target_nulls = df_raw[TARGET].isnull().sum()
print("Rows removed due to null spend_category:", removed_target_nulls)

df_raw = df_raw[df_raw[TARGET].notnull()].reset_index(drop=True)
print("Training shape after removing null targets:", df_raw.shape)


# =====================================================
# STEP 4 — MAIN PREPROCESSING FUNCTION
# =====================================================
def preprocess_raw_df(df):
    df = df.copy()

    # Clean strings
    for c in df.columns:
        if df[c].dtype == object:
            df[c] = df[c].astype(str).str.strip().str.rstrip(',')

    # Binary processing
    for c in binary_cols:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip().str.lower()
            df[c] = df[c].replace({
                "yes": 1,
                "no": 0,
                "nan": np.nan,
                "none": np.nan,
                "null": np.nan,
                "": np.nan
            })
            df[c] = df[c].fillna(0).astype(int)

    # Numeric count fields
    for c in numeric_count_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)

    # =====================================================
    # SAFE ORDINAL ENCODING FOR RANGE COLUMNS
    # =====================================================

    # Clean weird string values
    def clean_str(x):
        x = str(x).strip().lower()
        if x in ["nan", "none", "null", ""]:
            return np.nan
        return x

    # Clean raw string columns
    if "days_booked_before_trip" in df.columns:
        df["days_booked_before_trip_clean"] = df["days_booked_before_trip"].apply(clean_str)

    if "total_trip_days" in df.columns:
        df["total_trip_days_clean"] = df["total_trip_days"].apply(clean_str)

    # Define ordinal mappings
    ordinal_days_booked = {
        "1-7": 1,
        "8-14": 2,
        "15-30": 3,
        "31-60": 4,
        "61-90": 5,
        "90+": 6
    }

    ordinal_total_trip = {
        "1-6": 1,
        "7-14": 2,
        "15-30": 3,
        "30+": 4
    }

    # Map → Fill Missing → Convert to int
    if "days_booked_before_trip_clean" in df.columns:
        df["days_booked_before_trip_ord"] = (
            df["days_booked_before_trip_clean"]
                .map(ordinal_days_booked)
        )

        # Fill NaN with mode **of ordinal values**
        df["days_booked_before_trip_ord"].fillna(
            df["days_booked_before_trip_ord"].mode()[0],
            inplace=True
        )

        df["days_booked_before_trip_ord"] = df["days_booked_before_trip_ord"].astype(int)

    if "total_trip_days_clean" in df.columns:
        df["total_trip_days_ord"] = (
            df["total_trip_days_clean"]
                .map(ordinal_total_trip)
        )

        # Fill NaN with mode of ordinal values
        df["total_trip_days_ord"].fillna(
            df["total_trip_days_ord"].mode()[0],
            inplace=True
        )

        df["total_trip_days_ord"] = df["total_trip_days_ord"].astype(int)



    # Special requirements → binary
    if "has_special_requirements" in df.columns:
        df["has_special_req_bin"] = df["has_special_requirements"].astype(str).apply(
            lambda x: 0 if x.lower() in ["none", "", "nan"] else 1
        )

    return df


# =====================================================
# APPLY PREPROCESSING TO TRAIN & TEST
# =====================================================
df = preprocess_raw_df(df_raw).reset_index(drop=True)
test_df = preprocess_raw_df(test_raw).reset_index(drop=True)

print("\nAfter Base Preprocessing:")
print(df.shape)


# =====================================================
# STEP 5 — IMPUTATIONS (NO NaNs must remain)
# =====================================================

# Categorical mode fill
for c in categorical_cols:
    if c in df.columns:
        mode = df[c].mode()[0]
        df[c] = df[c].fillna(mode)
        test_df[c] = test_df[c].fillna(mode)

# =========================================================
# OUTLIER REMOVAL (TRAIN ONLY)
# And count number of rows removed per condition
# =========================================================

clean_df = df.copy()
initial_rows = len(clean_df)

outlier_info = {}

# ----- num_females ≤ 14 -----
before = len(clean_df)
clean_df = clean_df[clean_df["num_females"] <= 10]
after = len(clean_df)
outlier_info["num_females"] = before - after

# ----- Rule 1: num_males ≤ 13 -----
before = len(clean_df)
clean_df = clean_df[clean_df["num_males"] <= 10]
after_rule1 = len(clean_df)
removed_rule1 = before - after_rule1
before_rule2 = len(clean_df)

# Save results
outlier_info["num_males_threshold"] = removed_rule1
print("Removed (num_males > 20):", removed_rule1)


# ----- mainland_stay_nights ≤ 100 -----
before = len(clean_df)
clean_df = clean_df[clean_df["mainland_stay_nights"] <= 90]
after = len(clean_df)
outlier_info["mainland_stay_nights"] = before - after

# ----- island_stay_nights ≤ 21 -----
before = len(clean_df)
clean_df = clean_df[clean_df["island_stay_nights"] <= 60]
after = len(clean_df)
outlier_info["island_stay_nights"] = before - after

final_rows = len(clean_df)

# =========================================================
# PRINT OUTLIER REMOVAL SUMMARY
# =========================================================
print("\n========== OUTLIER REMOVAL SUMMARY ==========")
for col, removed in outlier_info.items():
    print(f"{col}: removed {removed} rows")

print("---------------------------------------------")
print(f"Total rows removed: {initial_rows - final_rows}")
print(f"Final Training Shape after Outlier Removal: {clean_df.shape}")
print("Test Shape (unchanged):", test_df.shape)


# =====================================================
# STEP 5 — FINAL FEATURE DEFINITIONS (No NaNs left)
# =====================================================

numeric_features = [
    "num_females",
    "num_males",
    "mainland_stay_nights",
    "island_stay_nights",
    "days_booked_before_trip_ord",
    "total_trip_days_ord"
]

binary_features = binary_cols + ["has_special_req_bin"]

categorical_features = categorical_cols

all_features = numeric_features + binary_features + categorical_features

clean_df = clean_df[all_features + [TARGET]]
test_df_final = test_df[all_features]




Original Training Shape: (12654, 25)
Original Test Shape: (5852, 24)
Rows removed due to null spend_category: 34
Training shape after removing null targets: (12620, 25)


/tmp/ipython-input-4121578799.py:92: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[c] = df[c].replace({
/tmp/ipython-input-4121578799.py:92: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[c] = df[c].replace({
/tmp/ipython-input-4121578799.py:92: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[c] =


After Base Preprocessing:
(12620, 30)
Removed (num_males > 20): 8

========== OUTLIER REMOVAL SUMMARY ==========
num_females: removed 28 rows
num_males_threshold: removed 8 rows
mainland_stay_nights: removed 27 rows
island_stay_nights: removed 8 rows
---------------------------------------------
Total rows removed: 71
Final Training Shape after Outlier Removal: (12549, 30)
Test Shape (unchanged): (5852, 29)


/tmp/ipython-input-4121578799.py:92: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[c] = df[c].replace({
/tmp/ipython-input-4121578799.py:92: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[c] = df[c].replace({
/tmp/ipython-input-4121578799.py:92: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[c] =

In [9]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

# =====================================================
# STEP X — FEATURE ENGINEERING: K-Means + GMM CLUSTERS
# =====================================================

print("\n=== Running Clustering Feature Engineering (KMeans) ===")

# ----------------------------
# 1. Prepare numeric data only
# ----------------------------
num_only = clean_df[numeric_features].copy()
test_num_only = test_df_final[numeric_features].copy()

# Standardize for clustering
scaler = StandardScaler()
num_scaled = scaler.fit_transform(num_only)
test_num_scaled = scaler.transform(test_num_only)

# ----------------------------
# 2. K-Means Clustering
# ----------------------------
kmeans = KMeans(n_clusters=6, random_state=42, n_init=10)
clean_df["kmeans_cluster"] = kmeans.fit_predict(num_scaled)
test_df_final["kmeans_cluster"] = kmeans.predict(test_num_scaled)

# # ----------------------------
# # 3. Gaussian Mixture Model (GMM)
# # ----------------------------
# gmm = GaussianMixture(n_components=5, random_state=42)
# clean_df["gmm_cluster"] = gmm.fit_predict(num_scaled)
# test_df_final["gmm_cluster"] = gmm.predict(test_num_scaled)

print("Clustering features added: kmeans_cluster")

# Add to categorical features
categorical_features += ["kmeans_cluster"]

# Update full feature list
all_features = numeric_features + binary_features + categorical_features

# Important: keep only final columns
clean_df = clean_df[all_features + [TARGET]]
test_df_final = test_df_final[all_features]

print("\nFinal shape after clustering:", clean_df.shape)



=== Running Clustering Feature Engineering (KMeans) ===
Clustering features added: kmeans_cluster

Final shape after clustering: (12549, 25)


/tmp/ipython-input-1854590008.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_final["kmeans_cluster"] = kmeans.predict(test_num_scaled)


In [10]:
preprocess = ColumnTransformer([
    ("num", StandardScaler(), numeric_features),
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ("bin", "passthrough", binary_features)
])

In [11]:
# =====================================================
# CORRECT SPLITTING ACCORDING TO YOUR REQUIREMENT
# =====================================================

from sklearn.model_selection import train_test_split

# ----------- FULL DATASET -----------
full_df = clean_df.copy()

# =====================================================
# SPLIT A: 80% train, 10% val, 10% test
# =====================================================

# First: 80% train, 20% temp
train_80, temp_20 = train_test_split(
    full_df,
    test_size=0.20,
    stratify=full_df[TARGET],
    random_state=42
)

# Now split 20% temp → 10% val + 10% test
val_10, test_10 = train_test_split(
    temp_20,
    test_size=0.50,   # half of 20%
    stratify=temp_20[TARGET],
    random_state=42
)

print("Split A (80/10/10):")
print("train_80:", train_80.shape)
print("val_10:", val_10.shape)
print("test_10:", test_10.shape)


# =====================================================
# SPLIT B: 20% train, 40% val, 40% test
# =====================================================

# First: 20% train, 80% temp
train_20, temp_80 = train_test_split(
    full_df,
    test_size=0.80,
    stratify=full_df[TARGET],
    random_state=42
)

# Now split 80% temp → 40% val + 40% test
val_40, test_40 = train_test_split(
    temp_80,
    test_size=0.50,
    stratify=temp_80[TARGET],
    random_state=42
)

print("\nSplit B (20/40/40):")
print("train_20:", train_20.shape)
print("val_40:", val_40.shape)
print("test_40:", test_40.shape)


Split A (80/10/10):
train_80: (10039, 25)
val_10: (1255, 25)
test_10: (1255, 25)

Split B (20/40/40):
train_20: (2509, 25)
val_40: (5020, 25)
test_40: (5020, 25)


In [12]:
import optuna
from optuna.samplers import TPESampler


In [13]:
from sklearn.svm import SVC

def build_svm_model(kernel_type):
    # Multiclass SVM supports 'ovo' naturally
    return Pipeline([
        ("prep", preprocess),
        ("svm", SVC(
            kernel=kernel_type,
            probability=False,   # much faster
            decision_function_shape='ovo'
        ))
    ])


In [14]:
def svm_objective(trial, kernel_name, X_train, y_train, preprocess, X_val, y_val):

    if kernel_name == "linear":
        # hyperparameters for linear SVM
        C = trial.suggest_float("C", 0.0001, 10.0, log=True)

        model = Pipeline([
            ("prep", preprocess),
            ("svm", SVC(
                kernel="linear",
                C=C,
                decision_function_shape="ovo"
            ))
        ])

    elif kernel_name == "rbf":
        # hyperparameters for rbf SVM
        C = trial.suggest_float("C", 0.0001, 10.0, log=True)
        gamma = trial.suggest_float("gamma", 0.0001, 1.0, log=True)

        model = Pipeline([
            ("prep", preprocess),
            ("svm", SVC(
                kernel="rbf",
                C=C,
                gamma=gamma,
                decision_function_shape="ovo"
            ))
        ])

    # Train
    model.fit(X_train, y_train)

    # Validate
    preds = model.predict(X_val)
    acc = accuracy_score(y_val, preds)

    return acc


In [15]:
def tune_svm_bayes(kernel_name, X_train, y_train, X_val, y_val, preprocess):

    study = optuna.create_study(
        direction="maximize",
        sampler=TPESampler(seed=42)
    )

    study.optimize(
        lambda trial: svm_objective(
            trial, kernel_name,
            X_train, y_train, preprocess,
            X_val, y_val
        ),
        n_trials=30  # you can reduce to 20 for speed
    )

    print(f"Best params for {kernel_name}: {study.best_params}")
    print(f"Best accuracy: {study.best_value}")

    return study.best_params


In [16]:
def build_tuned_svm(kernel_name, best_params, preprocess):

    if kernel_name == "linear":
        model = Pipeline([
            ("prep", preprocess),
            ("svm", SVC(
                kernel="linear",
                C=best_params["C"],
                decision_function_shape="ovo"
            ))
        ])

    elif kernel_name == "rbf":
        model = Pipeline([
            ("prep", preprocess),
            ("svm", SVC(
                kernel="rbf",
                C=best_params["C"],
                gamma=best_params["gamma"],
                decision_function_shape="ovo"
            ))
        ])

    return model


In [17]:

# =====================================================
# STEP 6 — FUNCTION TO MAKE SKEWED AND NON-SKEWED
# =====================================================
def make_skewed(df):
    # keep natural distribution
    return df.copy()

def make_nonskewed(df):
    # upsample minority classes
    major_size = df[TARGET].value_counts().max()
    frames = []
    for cls in df[TARGET].unique():
        cls_df = df[df[TARGET]==cls]
        cls_up = resample(cls_df, replace=True, n_samples=major_size, random_state=42)
        frames.append(cls_up)
    return pd.concat(frames)




In [18]:
def run_svm_and_save(mode_name, kernel_name, train_df, val_df, folder):
    os.makedirs(folder, exist_ok=True)

    X_train = train_df[all_features]
    y_train = train_df[TARGET]

    X_val = val_df[all_features]
    y_val = val_df[TARGET]

    print(f"\n🚀 Training SVM ({kernel_name}) for mode: {mode_name}")

    model = build_svm_model(kernel_name)
    model.fit(X_train, y_train)

    # predictions
    val_pred = model.predict(X_val)
    val_acc = accuracy_score(y_val, val_pred)

    # save classification report
    with open(f"{folder}/classification_report.txt", "w") as f:
        f.write(classification_report(y_val, val_pred))

    # save accuracy summary
    with open(f"{folder}/accuracy_summary.txt", "w") as f:
        f.write(f"Validation Accuracy: {val_acc}\n")

    # TEST predictions (final dataset)
    test_pred = model.predict(test_df_final)

    pd.DataFrame({
        ID_COL: test_df[ID_COL],
        TARGET: test_pred
    }).to_csv(f"{folder}/predictions_{kernel_name}_svm.csv", index=False)

    print(f"✅ Completed: {mode_name} — SVM {kernel_name}\n")


In [19]:
BASE = "/content/drive/MyDrive/ML_Project/MultinomialClassification/SVM/SVM_Tuning"
os.makedirs(BASE, exist_ok=True)

tasks = [
    ("80_skewed",     make_skewed(train_80),     val_10),
    ("80_nonskewed",  make_nonskewed(train_80),  val_10),
    ("20_skewed",     make_skewed(train_20),     val_40),
    ("20_nonskewed",  make_nonskewed(train_20),  val_40),
]

kernels = ["linear", "rbf", "poly", "sigmoid"]


In [20]:
for mode_name, tr_df, val_df in tasks:

    print(f"\n==============================")
    print(f"   MODE: {mode_name}")
    print(f"==============================\n")

    X_train = tr_df[all_features]
    y_train = tr_df[TARGET]

    X_val = val_df[all_features]
    y_val = val_df[TARGET]

    # ---------------------------------------------
    # 🔵 1. LINEAR SVM Bayesian tuning
    # ---------------------------------------------
    print(f"🔥 Bayesian tuning Linear SVM for {mode_name}")
    best_lin = tune_svm_bayes("linear", X_train, y_train, X_val, y_val, preprocess)

    lin_model = build_tuned_svm("linear", best_lin, preprocess)
    lin_model.fit(X_train, y_train)

    folder = f"{BASE}/{mode_name}/linear_svm_tuned"
    os.makedirs(folder, exist_ok=True)

    val_pred = lin_model.predict(X_val)
    test_pred = lin_model.predict(test_df_final)

    pd.DataFrame({
        ID_COL: test_df[ID_COL],
        TARGET: test_pred
    }).to_csv(f"{folder}/linear_svm_tuned_predictions.csv", index=False)

    with open(f"{folder}/val_report.txt", "w") as f:
        f.write(classification_report(y_val, val_pred))


    # ---------------------------------------------
    # 🔵 2. RBF SVM Bayesian tuning
    # ---------------------------------------------
    print(f"🔥 Bayesian tuning RBF SVM for {mode_name}")
    best_rbf = tune_svm_bayes("rbf", X_train, y_train, X_val, y_val, preprocess)

    rbf_model = build_tuned_svm("rbf", best_rbf, preprocess)
    rbf_model.fit(X_train, y_train)

    folder = f"{BASE}/{mode_name}/rbf_svm_tuned"
    os.makedirs(folder, exist_ok=True)

    val_pred = rbf_model.predict(X_val)
    test_pred = rbf_model.predict(test_df_final)

    pd.DataFrame({
        ID_COL: test_df[ID_COL],
        TARGET: test_pred
    }).to_csv(f"{folder}/rbf_svm_tuned_predictions.csv", index=False)

    with open(f"{folder}/val_report.txt", "w") as f:
        f.write(classification_report(y_val, val_pred))

    print(f"✔ Completed Bayesian SVM for: {mode_name}")



   MODE: 80_skewed



[I 2025-11-26 20:46:57,388] A new study created in memory with name: no-name-d9f05cb8-8751-40ed-a8c9-837aab475227


🔥 Bayesian tuning Linear SVM for 80_skewed


[I 2025-11-26 20:47:07,353] Trial 0 finished with value: 0.7211155378486056 and parameters: {'C': 0.0074593432857265485}. Best is trial 0 with value: 0.7211155378486056.
[I 2025-11-26 20:47:32,109] Trial 1 finished with value: 0.7346613545816733 and parameters: {'C': 5.669849511478847}. Best is trial 1 with value: 0.7346613545816733.
[I 2025-11-26 20:47:43,418] Trial 2 finished with value: 0.7386454183266933 and parameters: {'C': 0.4570563099801455}. Best is trial 2 with value: 0.7386454183266933.
[I 2025-11-26 20:47:52,939] Trial 3 finished with value: 0.7378486055776893 and parameters: {'C': 0.09846738873614563}. Best is trial 2 with value: 0.7386454183266933.
[I 2025-11-26 20:48:04,798] Trial 4 finished with value: 0.7243027888446215 and parameters: {'C': 0.0006026889128682511}. Best is trial 2 with value: 0.7386454183266933.
[I 2025-11-26 20:48:16,974] Trial 5 finished with value: 0.7243027888446215 and parameters: {'C': 0.000602521573620386}. Best is trial 2 with value: 0.73864541

Best params for linear: {'C': 0.6538211714622152}
Best accuracy: 0.7394422310756972


...

Best params for linear: {'C': 0.3470266988650412}
Best accuracy: 0.703386454183267


[I 2025-11-26 21:38:03,212] A new study created in memory with name: no-name-ba31a1ab-a20a-420a-85ec-cf3e7bbd62e4


🔥 Bayesian tuning RBF SVM for 20_nonskewed


[I 2025-11-26 21:38:08,729] Trial 0 finished with value: 0.5796812749003984 and parameters: {'C': 0.0074593432857265485, 'gamma': 0.6351221010640696}. Best is trial 0 with value: 0.5796812749003984.
[I 2025-11-26 21:38:12,624] Trial 1 finished with value: 0.6872509960159362 and parameters: {'C': 0.4570563099801455, 'gamma': 0.024810409748678097}. Best is trial 1 with value: 0.6872509960159362.
[I 2025-11-26 21:38:18,032] Trial 2 finished with value: 0.5055776892430279 and parameters: {'C': 0.0006026889128682511, 'gamma': 0.0004207053950287938}. Best is trial 1 with value: 0.6872509960159362.
[I 2025-11-26 21:38:23,436] Trial 3 finished with value: 0.548804780876494 and parameters: {'C': 0.00019517224641449495, 'gamma': 0.29154431891537513}. Best is trial 1 with value: 0.6872509960159362.
[I 2025-11-26 21:38:27,989] Trial 4 finished with value: 0.6766932270916335 and parameters: {'C': 0.10129197956845731, 'gamma': 0.06796578090758151}. Best is trial 1 with value: 0.6872509960159362.
[I 

Best params for rbf: {'C': 0.611835769263483, 'gamma': 0.1804261112705058}
Best accuracy: 0.7175298804780876
✔ Completed Bayesian SVM for: 20_nonskewed
